# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [9]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import math
%matplotlib inline
%matplotlib qt

In [7]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [66]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def RGB2HLS(img):
    # Convert image of RGB into HLS image
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

def Threshold(img,lowthreshold=40,highthreshold=255):
    # Creating a Binary image using S channel and its threshold values
    return cv2.inRange(img,lowthreshold,highthreshold)
    
def canny(img, low_threshold=50, high_threshold=150):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

# I tried this function but it is not working that good 
# Reference: http://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
 
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
 
    # return the edged image
    return edged

def gaussian_blur(img, kernel_size=7):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img):
    """
    Applies an image mask.
    """
    hei, wid = img.shape
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    ignore_mask_color = 255
    # creating outer mask: 
    vertices = np.array([[0.45 * wid, 0.6 * hei], 
                         [0.55 * wid, 0.6 * hei],
                         [       wid,        hei],
                         [ 0.1 * wid,        hei]], np.int32)
    cv2.fillConvexPoly(mask, vertices, ignore_mask_color)

    # creating inner mask:
    vertices = np.array([[0.5 * wid, 0.6 * hei],
                         [0.75 * wid,      hei],
                         [0.25 * wid,      hei]], np.int32)
    cv2.fillConvexPoly(mask, vertices, 0)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if lines is None:
        return
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y1 - y2) < 30: #ignore nearly-horizontal lines
                continue
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
          

        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros(img.shape, dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1.0, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


    

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [71]:
def pipeline(image):
    # TODO: Build your pipeline that will draw lane lines on the test_images
    # then save them to the test_images directory.
    gray = grayscale(image)
    #color_select = np.copy(image)
    # Converting to HLS color
    #hls = RGB2HLS(color_select)
    #S = hls[:,:,2]
    #threshold = (40,255)
    # Using S channel we will create a Binary channel using cv2.inRange
    #Binary = Threshold(S,threshold[0],threshold[1])
    #(M,N) = Binary.shape 
    # Gaussian Blur
    kernel_size = 7
    image_gaussian = gaussian_blur(gray,kernel_size)
    # Canny Edge Detection
    low_threshold = 50
    high_threshold = 150
    image_Edges = canny(image_gaussian,low_threshold,high_threshold)
    # Region of Interest
    #masking images
    masked_Image = region_of_interest(image_Edges)
    black = np.zeros_like(masked_Image)
    #Hough Transform
    lines_s = hough_lines(masked_Image, 0.1, np.pi/360, 10, 60, 30)
    lines_m = hough_lines(masked_Image, 0.1, np.pi/360, 10, 120, 80)
    lines_l = hough_lines(masked_Image, 0.1, np.pi/260, 20, 180, 100)
    lines_all = lines_s + lines_m + lines_l
    lines = np.dstack((lines_all, black, black))
    #print(lines.shape)
    #print(image.shape)
    combo = weighted_img(lines.astype("uint8"), image)
    return combo

images = os.listdir("test_images/")
# Frame 1
img1 = mpimg.imread('test_images/'+images[1])
img1_Lane = pipeline(img1)
plt.figure()
plt.title(images[0])
plt.imshow(img1_Lane)


run your solution on all test_images and make copies into the test_images directory).

In [75]:
for imgname in images:
    img = mpimg.imread("test_images/"+imgname)
    processed_img = pipeline(img)
    # print('This image is: ', type(image), 'with dimensions:', image.shape)
    fig = plt.figure(figsize=(10,3))
    fig.suptitle(imgname, fontsize=15)
    ax = plt.subplot(121)
    ax.set_title("Before")
    plt.imshow(img)
    ax = plt.subplot(122)
    ax.set_title("After")
    plt.imshow(processed_img, cmap='gray')

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [76]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    # call your pipeline
    result = pipeline(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [77]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

100%|█████████▉| 221/222 [00:30<00:00,  9.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 15.5 s, sys: 5.36 s, total: 20.9 s
Wall time: 31.8 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [39]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [40]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|          | 4/682 [00:00<00:19, 35.44it/s]

  1%|          | 7/682 [00:00<00:21, 31.42it/s]

  1%|▏         | 10/682 [00:00<00:22, 30.05it/s]

  2%|▏         | 14/682 [00:00<00:22, 30.06it/s]

  3%|▎         | 18/682 [00:00<00:21, 30.64it/s]

  3%|▎         | 21/682 [00:00<00:22, 30.00it/s]

  4%|▎         | 24/682 [00:00<00:22, 29.76it/s]

  4%|▍         | 27/682 [00:00<00:22, 29.32it/s]

  4%|▍         | 30/682 [00:01<00:22, 29.41it/s]

  5%|▍         | 33/682 [00:01<00:24, 26.78it/s]

  5%|▌         | 36/682 [00:01<00:23, 27.31it/s]

  6%|▌         | 39/682 [00:01<00:44, 14.54it/s]

  6%|▌         | 42/682 [00:01<00:46, 13.70it/s]

  6%|▋         | 44/682 [00:02<00:50, 12.73it/s]

  7%|▋         | 46/682 [00:02<00:47, 13.46it/s]

  7%|▋         | 49/682 [00:03<01:41,  6.25it/s]

  7%|▋         | 51/682 [00:04<03:32,  2.97it/s]

  8%|▊         | 52/682 [00:05<03:25,  3.06it/s]

  8%|▊         | 54/682 [00:05<02:37,  4.00it/s]

  8%|▊   

[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 18.2 s, sys: 572 ms, total: 18.8 s
Wall time: 57.6 s


In [79]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Reflections
I would like to thank sedarolmez for the positive feedback and the explaining the need changes in the values and why for hough lines.
Reflection describes the current pipeline, identifies its potential shortcomings and suggests possible improvements. 
The current pipeline consists of converting image into grayscale image. The grayscale is blurred using gaussian filter.  A region of interest function for selecting desired region in image for lanes.Applying hough transform. Here to increase the confidence of hough lines i have created three hough lines parameter and added them. 
I have taken the idea for region of interest and draw_lines function from "https://github.com/ryan-keenan/CarND"

I initially debugged the pipeline with S channel of the image rather grayscale image. But theperformance of  pipeline is not good. 

As we can see from challenge video, The pipeline is not working for images with steep curves and shadows.
Future work: Need to explore the colorspace, implement convolution method explained in Q/A session of SelfDrivingCar - Advanced Lane Lines.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [80]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

  self.nchannels))



[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:50<00:00,  5.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 22.8 s, sys: 15.7 s, total: 38.5 s
Wall time: 53.5 s


In [81]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))